# *Bag of visual words*

Experimentos para construir un "[vocabulario visual](https://en.wikipedia.org/wiki/Bag-of-words_model_in_computer_vision)".

## Bibliotecas y funciones auxiliares

In [ ]:
import numpy             as np
import cv2               as cv
import matplotlib.pyplot as plt
import glob
import pickle

def fig(w,h):
    plt.figure(figsize=(w,h))

def readrgb(file):
    return cv.cvtColor( cv.imread("../images/"+file), cv.COLOR_BGR2RGB) 
    
import glob
def readfiles(path):
    return [readrgb(file) for file in sorted(glob.glob('../images/'+path))]

## wikiart

Nuestro objetivo en este caso de estudio es identificar pinturas de Velázquez a partir de fragmentos. La página web [wikiart](https://www.wikiart.org/) proporciona imágenes de gran calidad que podemos descargar mediante la utilidad [wikiart retriever](https://github.com/lucasdavid/wikiart).

In [ ]:
imgs = readfiles('velazquez/*/*.jpg')
print(len(imgs))

Las imágenes se caracterizarán mediante sus puntos SIFT:

In [ ]:
sift = cv.SIFT_create(nfeatures=0, contrastThreshold = 0.07)

img = imgs[5]

view = img.copy()

kp,desc = sift.detectAndCompute(img, mask=None)

flag = cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
cv.drawKeypoints(view,kp,view, color=(100,150,255), flags=flag)

fig(15,10)
print(len(kp))
plt.imshow(view);

Lo primero que hacemos es calcular los descriptores SIFT de toda la colección y guardarlos en memoria, de modo que en lo sucesivo podamos recuperarlos rápidamente. Guardamos los de cada imagen por separado en `allpoints`, y la colección completa en el array `points`. (Es redundante, pero así ilustramos la forma de trabajar con `pickle` y con `np.save_compressed`).

In [ ]:
def mkSIFT(nfeatures=0, contrastThreshold=0.04, minscale=0):
        sift = cv.SIFT_create(nfeatures=nfeatures, contrastThreshold = contrastThreshold)
        def fun(x):
            kp,desc = sift.detectAndCompute(x, mask=None)
            sc = np.array([k.size for k in kp])
            return desc[sc>minscale].astype(np.uint8)
        return fun


sift = mkSIFT(contrastThreshold = 0.07)

if False:
    allpoints = []
    for k,x in enumerate(imgs):
        allpoints.append(sift(x))
        print(k, len(allpoints[-1]))

    pickle.dump( allpoints, open( "allpoints.p", "wb" ) )
    points = np.vstack(allpoints)
    np.savez_compressed('keypoints', points=points)
    
else:
    allpoints = pickle.load( open( "../data/allpoints.p", "rb" ) )
    points = np.load('../data/keypoints.npz')['points'].astype(np.float32)

points.shape, points.dtype

Por curiosidad, mostramos la distribución de simulitudes de un cierto punto con todos los demás.

In [ ]:
dis = abs(points - points[2334]).sum(axis=1)/128

plt.hist(dis,bins=30);# plt.ylim(0,200)

## *Matching* "normal" 

En primer lugar vamos a intentar reconocer las obras mediante el número de coincidencias "buenas", igual que en ejemplo simple del notebook [keypoints](keypoints.ipynb).

In [ ]:
bf = cv.BFMatcher()

# número de coincidencias que superan el ratio test
def match(query, model):
    matches = bf.knnMatch(query,model,k=2)
    # ratio test
    good = []
    for m in matches:
        if len(m) == 2:
            best, second = m
            if best.distance < 0.75*second.distance:
                good.append(best)
    return len(good)

# devuelve una lista ordenada de número de matchings-índice del modelo
def find(x):
    v = sift(x)
    print(len(v))
    dists = sorted([(match(v,u),k) for k,u in enumerate(allpoints)])[::-1]
    return dists

Por ejemplo, la primera obra coincide con ella misma en 156 puntos, con la de índice 29 solo en 12, etc.

In [ ]:
find(imgs[0])[:20]

Hemos preparado unos cuantos recortes de algunas obras para probar el método.

In [ ]:
cosas = readfiles('sift/1640/*.*')

La primera de ellas tiene 647 puntos SIFT, de los cuales coinciden 539 con el modelo 86, y el resto de modelos tiene muchas menos.

In [ ]:
find(cosas[0])[:10]

Vamos aponerlo más difícil, rotando, reduciendo de tamaño y suavizando el fragmento:

In [ ]:
%%time

k = 2
b = 0
fig(12,6)
plt.subplot(1,2,1)
orig = cosas[k]
obs  = np.rot90(cv.GaussianBlur(cv.resize(orig,(0,0), fx=0.7, fy=0.7),(0,0), 2),1)
plt.imshow(obs);
dists = find(obs)[:5]
print(f'{dists[0][0]} - {dists[1][0]}')
best = dists[b][1]
plt.subplot(1,2,2)
plt.imshow(imgs[best]);

La evaluación es muy rápida debido a que en este caso en la imagen desconocida hay pocos puntos SIFT (35), de los cuales 17 coinciden con un modelo, que resulta ser correcto. El segundo mejor solo tiene 2 coincidencias.

Cuando la imagen tiene más puntos el tiempo de cómputo empieza a ser elevado:

In [ ]:
%%time

k = 8
b = 0
fig(12,6)
plt.subplot(1,2,1)
orig = cosas[k]
obs  = np.rot90(cv.GaussianBlur(cv.resize(orig,(0,0), fx=1.2, fy=1.2),(0,0), 0.1),1)
plt.imshow(obs);
dists = find(obs)[:5]
print(f'{dists[0][0]} - {dists[1][0]}')
best = dists[b][1]
plt.subplot(1,2,2)
plt.imshow(imgs[best]);

## k-means

Para intentar acelerar el tiempo de detección vamos a construir un "vocabulario visual" agrupando los puntos SIFT en un conjunto de, por ejemplo, 500 tipos. 

In [ ]:
import pickle


from sklearn.cluster import KMeans
#from sklearn.externals import joblib

El proceso require varios minutos (dependiendo del ordenador, del número de puntos y de categorías puede superar media hora), por lo que lo almacenamos.

In [ ]:
%%time

if False:
    #codebook = KMeans(n_clusters=500, random_state=0).fit(points[np.random.choice(len(points), 100000)])
    codebook = KMeans(n_clusters=500, random_state=0).fit(points)
    with open('codebook.pkl', 'wb') as handle:
        pickle.dump(codebook, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open('../data/codebook.pkl', 'rb') as handle:
        codebook = pickle.load(handle)

In [ ]:
# to fix a strange error
codebook.cluster_centers_ = codebook.cluster_centers_.astype(float)

Los descriptores de cada punto SIFT se sustituyen por la etiqueta del *cluster* más próximo:

In [ ]:
codebook.predict(sift(imgs[1]))

Por curiosidad, mostramos la distribución de distancias de cada punto a su cluster.

In [ ]:
desc = sift(imgs[1])
index = codebook.predict(desc)
r = codebook.cluster_centers_[index] - desc
d = np.sqrt((r**2).sum(axis=1))
plt.hist(d);

Las imágenes se representarán mediante el histograma de códigos de descriptores. (Teniendo en cuenta solo aquellos que están suficientemente cerca del cluster asignado. No está claro si esto tiene influencia positiva.)

In [ ]:
def getcode(x):
    desc = sift(x)
    index = codebook.predict(desc)
    r = codebook.cluster_centers_[index] - desc
    d = np.sqrt((r**2).sum(axis=1))
    return np.histogram(index[d<250],np.arange(codebook.n_clusters+1))[0]

Veamos el histograma de un par de imágenes:

In [ ]:
plt.plot(getcode(imgs[1]));

In [ ]:
plt.plot(getcode(imgs[35]));

Calculamos los histogramas de toda la colección y los almacenamos.

In [ ]:
%%time

if False:
    imagecodes = [getcode(x) for x in imgs]
    pickle.dump( imagecodes, open( "imagecodes.p", "wb" ) )
else:
    imagecodes = pickle.load( open( "../data/imagecodes.p", "rb" ) )

Para comparar este tipo de histogramas la similitud la medimos con la suma de mínimos en cada caja (intersección). La idea es que para que haya una coincidencia, los puntos deberían ir a la misma caja del histograma. Se normaliza el valor con el número total de puntos, intentando potenciar que se cubra lo más posible el modelo.

In [ ]:
def simil(u,v):
    t = max(u.sum(),v.sum())
    return np.minimum(u,v).sum()/t


def find(x):
    v = getcode(x)
    print(v.sum())
    dists = sorted([(simil(v,u),k) for k,u in enumerate(imagecodes)])[::-1]
    return dists

Como primera prueba, vemos la similitud de uno de los modelos con toda la colección. El mejor es él mismo, con similitud perfecta (1.0) y el siguiente queda muy lejos.

In [ ]:
find(imgs[1])[:10]

Esta misma información en forma de histograma:

In [ ]:
plt.hist([x[0] for x in find(imgs[1])]);

Sin embargo, cuando modificamos bastante la imagen, las distancias se igualan mucho. Y en este caso concreto la imagen correcta queda en posición 9...

In [ ]:
orig = imgs[50]
obs  = cv.GaussianBlur(cv.resize(orig,(0,0), fx=0.7, fy=0.7),(0,0), 2)
print(simil(getcode(orig),getcode(obs)))
dists = find(obs)
plt.hist([x[0] for x in dists])
find(obs)[:15]

La mayor similitud no siempre corresponde con el modelo correcto. Lo importante es que éste quede en las primeras posiciones, de modo que podamos aplicar la técnica de coincidencias más precisa sólo con los, p. ej. 20, mejores candidatos, en lugar de los más de 100 modelos de toda la colección.

In [ ]:
k = 0
b = 0
fig(10,5)
plt.subplot(1,2,1)
plt.imshow(cosas[k]);
dists = find(cosas[k])
print(f'{dists[0][0]:.2f} - {dists[1][0]:.2f}')
best = dists[b][1]
plt.subplot(1,2,2)
plt.imshow(imgs[best]);

In [ ]:
plt.hist([x[0] for x in dists]);